### Evaluate Topic Model in Python: Latent Dirichlet Allocation (LDA)

In the previous article, I introduced the concept of topic modeling and walked through the code for developing your first topic model using Latent Dirichlet Allocation (LDA) method in the python using sklearn implementation.

Pursuing on that understanding, in this article, I'll go a few steps deeper by outlining the framework to quantitatively evaluate topic models through the measure of topic coherence and share the code template in python using Gensim implementation to allow for end-to-end model development.

### Why evaluate topic models?

![img](https://tinyurl.com/y3xznjwq)

We know probabilistic topic models, such as LDA, are popular tools for analysis of the text, providing both a predictive and latent topic representation of the corpus. There is a longstanding assumption that the latent space discovered by these models is meaningful and useful, and evaluating such assumptions is challenging due to its unsupervised training process. There is a no-gold standard list of topics to compare against every corpus.

However, it is equally important to identify if a trained model is objectively good or bad, as well have an ability to compare different models/methods and to do so, we require an objective measure for the quality. Traditionally, and still for many practical applications, to evaluate if "the correct thing" has been learned about the corpus, we use implicit knowledge and "eyeballing" approaches. Ideally, we'd like to capture this information in a single metric that can be maximized, and compared. Let's take a look at roughly what approaches are commonly used for the evaluation:

**Eye Balling Models**
- Top N words
- Topics / Documents

**Intrinsic Evaluation Metrics**
- Capturing model semantics
- Topics interpretability

**Human Judgements**
- What is a topic

**Extrinsic Evaluation Metrics/Evaluation at task**
- Is model good at performing predefined tasks, such as classification

Natural language is messy, ambiguous and full of subjective interpretation, and sometimes trying to cleanse ambiguity reduces the language to an unnatural form. Nevertheless, in this article, we'll explore more about topic coherence, and how we can use it to quantitatively justify the model selection.

### What is Topic Coherence?

Perplexity is often used as an example of an intrinsic evaluation measure. It comes from the language modeling community and aims to capture how surprised a model is of new data it has not seen before. It is measured as the normalized log-likelihood of a held-out test set.

Focussing on the log-likelihood part, you can think of the perplexity metric as measuring how probable some new unseen data is given the model that was learned earlier. That is to say, how well does the model represent or reproduce the statistics of the held-out data.

However, past research has shown that predictive likelihood (or equivalently, perplexity) and human judgment are often not correlated, and even sometimes slightly anti-correlated. And that served as a motivation for more work trying to model the human judgment, and thus `Topic Coherence`.

The topic coherence concept combines a number of papers into one framework that allows evaluating the coherence of topics inferred by a topic model. But,

#### What is topic coherence?
Topic Coherence measures score a single topic by measuring the degree of semantic similarity between high scoring words in the topic. These measurements help distinguish between topics that are semantically interpretable topics and topics that are artifacts of statistical inference. But,

#### What is coherence?
A set of statements or facts is said to be coherent, if they support each other. Thus, a coherent fact set can be interpreted in a context that covers all or most of the facts. An example of a coherent fact set is "the game is a team sport", "the game is played with a ball", "the game demands great physical efforts"

### Coherence Measures

1. `C_v` measure is based on a sliding window, one-set segmentation of the top words and an indirect confirmation measure that uses normalized pointwise mutual information (NPMI) and the cosine similarity
2. `C_p` is based on a sliding window, one-preceding segmentation of the top words and the confirmation measure of Fitelson's coherence
3. `C_uci` measure is based on a sliding window and the pointwise mutual information (PMI) of all word pairs of the given top words
4. `C_umass` is based on document cooccurrence counts, a one-preceding segmentation and a logarithmic conditional probability as confirmation measure
5. `C_npmi` is an enhanced version of the C_uci coherence using the normalized pointwise mutual information (NPMI)
6. `C_a` is baseed on a context window, a pairwise comparison of the top words and an indirect confirmation measure that uses normalized pointwise mutual information (NPMI) and the cosine similarity

### Model Implementation

#### Loading data

For this tutorial, we’ll use the dataset of papers published in NIPS conference. The NIPS conference (Neural Information Processing Systems) is one of the most prestigious yearly events in the machine learning community. The CSV data file contains information on the different NIPS papers that were published from 1987 until 2016 (29 years!). These papers discuss a wide variety of topics in machine learning, from neural networks to optimization methods, and many more.

<img src="https://s3.amazonaws.com/assets.datacamp.com/production/project_158/img/nips_logo.png" alt="The logo of NIPS (Neural Information Processing Systems)">

Let’s start by looking at the content of the file

In [1]:
# Importing modules
import pandas as pd
import os

os.chdir('..')

# Read data into papers
papers = pd.read_csv('./data/NIPS Papers/papers.csv')

# Print head
papers.head()

,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a..."


#### Data Cleaning

Since the goal of this analysis is to perform topic modeling, we will solely focus on the text data from each paper, and drop other metadata columns

In [2]:
# Remove the columns
papers = papers.drop(columns=['id', 'title', 'abstract', 
                              'event_type', 'pdf_name', 'year'], axis=1)

# sample only 100 papers
papers = papers.sample(100)

# Print out the first rows of papers
papers.head()

,paper_text
1454,Optimality of Reinforcement Learning\nAlgorith...
3539,Dynamic Pooling and Unfolding Recursive\nAutoe...
4758,Greedy Subspace Clustering\nDohyung Park\nDept...
3752,Semi-supervised Regression via\nParallel Field...
4599,Binary to Bushy: Bayesian Hierarchical Cluster...


#### Remove punctuation/lower casing

Next, let’s perform a simple preprocessing on the content of paper_text column to make them more amenable for analysis, and reliable results. To do that, we’ll use a regular expression to remove any punctuation, and then lowercase the text

In [3]:
# Load the regular expression library
import re

# Remove punctuation
papers['paper_text_processed'] = papers['paper_text'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
papers['paper_text_processed'] = papers['paper_text_processed'].map(lambda x: x.lower())

# Print out the first rows of papers
papers['paper_text_processed'].head()

1454    optimality of reinforcement learning\nalgorith...
3539    dynamic pooling and unfolding recursive\nautoe...
4758    greedy subspace clustering\ndohyung park\ndept...
3752    semi-supervised regression via\nparallel field...
4599    binary to bushy: bayesian hierarchical cluster...
Name: paper_text_processed, dtype: object

#### Tokenize words and further clean-up text

Let’s tokenize each sentence into a list of words, removing punctuations and unnecessary characters altogether.

In [4]:
import gensim
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = papers.paper_text_processed.values.tolist()
data_words = list(sent_to_words(data))

print(data_words[:1][0][:30])

[u'optimality', u'of', u'reinforcement', u'learning', u'algorithms', u'with', u'linear', u'function', u'approximation', u'ralf', u'schoknecht', u'ilkd', u'university', u'of', u'karlsruhe', u'germany', u'ralfschoknecht', u'ilkduni', u'karlsruhede', u'abstract', u'there', u'are', u'several', u'reinforcement', u'learning', u'algorithms', u'that', u'yield', u'approximate', u'solutions']


#### Creating Bigram and Trigram Models

Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring. Some examples in our example are: 'back_bumper', 'oil_leakage', 'maryland_college_park' etc.

Gensim's Phrases model can build and implement the bigrams, trigrams, quadgrams and more. The two important arguments to Phrases are min_count and threshold.

In [5]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

#### Remove Stopwords, Make Bigrams and Lemmatize

The phrase models are ready. Let’s define the functions to remove the stopwords, make trigrams and lemmatization and call them sequentially.

In [6]:
# NLTK Stop words
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

Let's call the functions in order.

In [8]:
import spacy

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

[u'optimality', u'reinforcement_learne', u'algorithm', u'linear', u'function', u'approximation', u'ralf', u'schoknecht', u'ilkd', u'university', u'karlsruhe', u'germany', u'ralfschoknecht', u'ilkduni', u'karlsruhede', u'abstract', u'several', u'reinforcement_learning', u'algorithm', u'yield', u'approximate', u'solution', u'problem', u'policy', u'evaluation', u'value', u'function', u'represent', u'linear', u'function']


### Data transformation: Corpus and Dictionary

The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus. Let’s create them.

In [9]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 9), (5, 1), (6, 1), (7, 8), (8, 1), (9, 2), (10, 2), (11, 1), (12, 1), (13, 14), (14, 1), (15, 1), (16, 1), (17, 55), (18, 2), (19, 1), (20, 4), (21, 1), (22, 1), (23, 2), (24, 1), (25, 3), (26, 1), (27, 1), (28, 2), (29, 5)]


### Building the base topic model

We have everything required to train the base LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well. Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior (we'll use default for the base model).

chunksize controls how many documents are processed at a time in the training algorithm. Increasing chunksize will speed up training, at least as long as the chunk of documents easily fit into memory.

passes controls how often we train the model on the entire corpus (set to 10). Another word for passes might be "epochs". iterations is somewhat technical, but essentially it controls how often we repeat a particular loop over each document. It is important to set the number of "passes" and "iterations" high enough.

In [10]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

#### View the topics in LDA model
The above LDA model is built with 10 different topics where each topic is a combination of keywords and each keyword contributes a certain weightage to the topic.

You can see the keywords for each topic and the weightage(importance) of each keyword using `lda_model.print_topics()`

In [11]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  u'0.013*"algorithm" + 0.010*"state" + 0.009*"set" + 0.008*"point" + 0.008*"subspace" + 0.008*"cluster" + 0.007*"tree" + 0.006*"use" + 0.006*"function" + 0.006*"node"'),
 (1,
  u'0.016*"sample" + 0.010*"algorithm" + 0.009*"distribution" + 0.008*"mixture" + 0.007*"estimator" + 0.007*"time" + 0.007*"machine" + 0.006*"tree" + 0.006*"give" + 0.006*"log"'),
 (2,
  u'0.012*"neuron" + 0.008*"control" + 0.007*"use" + 0.007*"time" + 0.006*"optimal" + 0.006*"spike" + 0.006*"model" + 0.006*"reward" + 0.005*"system" + 0.005*"change"'),
 (3,
  u'0.010*"function" + 0.010*"view" + 0.009*"learn" + 0.009*"datum" + 0.007*"problem" + 0.006*"show" + 0.005*"use" + 0.005*"approach" + 0.005*"result" + 0.005*"model"'),
 (4,
  u'0.011*"model" + 0.011*"function" + 0.009*"use" + 0.009*"policy" + 0.007*"datum" + 0.007*"method" + 0.007*"matrix" + 0.006*"algorithm" + 0.006*"problem" + 0.006*"kernel"'),
 (5,
  u'0.013*"model" + 0.010*"function" + 0.008*"set" + 0.007*"use" + 0.007*"learn" + 0.006*"algorithm" + 

#### Compute Model Perplexity and Coherence Score

Let's calculate the baseline coherence score

In [13]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

('Coherence Score: ', 0.30324211900240483)


### Hyperparameter tuning

First, let's differentiate between model hyperparameters and model parameters :

- `Model hyperparameters` can be thought of as settings for a machine learning algorithm that are tuned by the data scientist before training. Examples would be the number of trees in the random forest, or in our case, number of topics K

- `Model parameters` can be thought of as what the model learns during training, such as the weights for each word in a given topic.

Now that we have the baseline coherence score for the default LDA model, let's perform a series of sensitivity tests to help determine the following model hyperparameters: 
- Number of Topics (K)
- Dirichlet hyperparameter alpha: Document-Topic Density
- Dirichlet hyperparameter beta: Word-Topic Density

We'll perform these tests in sequence, one parameter at a time by keeping others constant and run them over the two difference validation corpus sets. We'll use `C_v` as our choice of metric for performance comparison 

In [14]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

Let's call the function, and iterate it over the range of topics, alpha, and beta parameter values

In [15]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, num_of_docs*0.75), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

100%|██████████| 540/540 [1:55:58<00:00, 15.09s/it]


### Final Model Training

Based on external evaluation (Code to be added from Excel based analysis), train the final model

In [23]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.01,
                                           eta=0.9)

In [24]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  u'0.006*"algorithm" + 0.006*"cluster" + 0.006*"set" + 0.005*"node" + 0.005*"state" + 0.004*"clustering" + 0.004*"category" + 0.003*"network" + 0.003*"use" + 0.003*"show"'),
 (1,
  u'0.008*"algorithm" + 0.006*"subspace" + 0.006*"sample" + 0.005*"point" + 0.004*"estimator" + 0.004*"mixture" + 0.004*"log" + 0.004*"machine" + 0.003*"cluster" + 0.003*"time"'),
 (2,
  u'0.006*"spike" + 0.004*"control" + 0.004*"neuron" + 0.003*"system" + 0.003*"optimal" + 0.003*"segment" + 0.003*"vor" + 0.003*"dynamic" + 0.002*"noise" + 0.002*"use"'),
 (3,
  u'0.007*"learn" + 0.006*"problem" + 0.006*"datum" + 0.006*"function" + 0.005*"set" + 0.005*"algorithm" + 0.005*"use" + 0.004*"model" + 0.004*"view" + 0.004*"give"'),
 (4,
  u'0.010*"function" + 0.008*"problem" + 0.007*"algorithm" + 0.007*"use" + 0.006*"model" + 0.006*"set" + 0.006*"policy" + 0.006*"state" + 0.005*"matrix" + 0.005*"method"'),
 (5,
  u'0.012*"model" + 0.007*"learn" + 0.006*"set" + 0.006*"function" + 0.006*"algorithm" + 0.006*"datum" 

In [28]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

LDAvis_prepared

/home/jupyter/.local/lib/python2.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
5      32.598530        1       1 -0.099315  0.031513
4      20.686474        1       2 -0.076826 -0.033969
7      19.285721        1       3 -0.061428  0.015668
3      12.964823        1       4 -0.039307 -0.023796
0       7.073448        1       5  0.017009  0.040609
1       4.464208        1       6  0.033080 -0.047669
2       2.440185        1       7  0.096261  0.025379
6       0.486616        1       8  0.130526 -0.007735, topic_info=      Category         Freq            Term        Total  loglift  logprob
6839   Default   233.000000          neuron   233.000000  30.0000  30.0000
2636   Default  1165.000000       algorithm  1165.000000  29.0000  29.0000
8238   Default   273.000000          policy   273.000000  28.0000  28.0000
8602   Default   257.000000         cluster   257.000000  27.0000  27.0000
8152   Default   519.000000          sample   519.000000  26.0000  26.0000
12740  Default    70.000000         circuit    70.000000  25.0000  25.0000
10725  Default    87.000000           spike    87.000000  24.0000  24.0000
6741   Default   638.000000           state   638.000000  23.0000  23.0000
6399   Default   132.000000         control   132.000000  22.0000  22.0000
1047   Default   400.000000         optimal   400.000000  21.0000  21.0000
6815   Default    93.000000        subspace    93.000000  20.0000  20.0000
11491  Default   410.000000            node   410.000000  19.0000  19.0000
9421   Default   441.000000           point   441.000000  18.0000  18.0000
2992   Default   325.000000          system   325.000000  17.0000  17.0000
184    Default   274.000000         machine   274.000000  16.0000  16.0000
2542   Default   168.000000      clustering   168.000000  15.0000  15.0000
6767   Default   834.000000         problem   834.000000  14.0000  14.0000
11084  Default   124.000000         mixture   124.000000  13.0000  13.0000
5265   Default   386.000000             log   386.000000  12.0000  12.0000
7118   Default   142.000000         dynamic   142.000000  11.0000  11.0000
10489  Default   184.000000           noise   184.000000  10.0000  10.0000
13780  Default   189.000000            view   189.000000   9.0000   9.0000
12274  Default    75.000000       estimator    75.000000   8.0000   8.0000
2936   Default   501.000000          matrix   501.000000   7.0000   7.0000
6308   Default   127.000000            cell   127.000000   6.0000   6.0000
1915   Default  1023.000000           learn  1023.000000   5.0000   5.0000
8835   Default   168.000000          vertex   168.000000   4.0000   4.0000
11113  Default   145.000000       detection   145.000000   3.0000   3.0000
181    Default   406.000000         feature   406.000000   2.0000   2.0000
7380   Default   215.000000        gradient   215.000000   1.0000   1.0000
...        ...          ...             ...          ...      ...      ...
11119   Topic8     0.413790   correlational     5.000957   2.8334  -7.7964
8446    Topic8     0.343192            shon     4.258848   2.8070  -7.9835
469     Topic8     0.343089        figueroa     4.258828   2.8067  -7.9838
11372   Topic8     0.343040             hsu     4.261018   2.8060  -7.9839
2255    Topic8     1.292789          analog    16.928625   2.7532  -6.6572
3575    Topic8     0.342775             cmo     5.012762   2.6428  -7.9847
6587    Topic8     0.342910       configure     5.085239   2.6288  -7.9843
9711    Topic8     0.272556   functionality     4.181179   2.5949  -8.2139
11836   Topic8     0.272508        designer     4.180912   2.5948  -8.2141
799     Topic8     0.272507            fpga     4.181030   2.5948  -8.2141
3941    Topic8     0.272419        nfet_plb     4.181927   2.5943  -8.2144
9999    Topic8     0.272384             dac     4.181856   2.5942  -8.2146
2421    Topic8     0.754333        comprise    11.857322   2.5706  -7.1959
12740   Topic8     3.775337         circui